In [1]:
# ใช้โค้ดเดิมใน spliting_fasta.ipynb
from Bio import SeqIO
import pandas as pd

def extractHLAfromFasta(filename):
    """ ดึงข้อมูล HLA ออกจาก fasta file แล้ว return list of HLA Alleles """
    HLAnomenclature_list = []
    for seq_record in SeqIO.parse(filename, "fasta"):  
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        HLAnomenclature_list.append(HLAnomenclature)
    return HLAnomenclature_list

def extractGenefromHLA(listofHLA):
    """ รับ list of HLA alleles, แล้ว return list of Gene"""
    gene_list = []
    for HLA in listofHLA:
        gene = HLA.split("*")[0]
        gene_list.append(gene)
    return gene_list

def returnuniquefromlist(list):
    """ รับ list ใด ๆ แล้ว return unique values of the list"""
    unique_list = []
    loaded_list = pd.DataFrame(list).drop_duplicates()
    for lab,row in loaded_list.iterrows():
        unique_list.append(row.item())
    return unique_list

def extractHLAfromGenelist(listofHLA, listofgene):
    """ รับ list of gene แล้ว return list of HLA ที่มียีนนั้น """
    return_list = []
    for HLA in listofHLA:
        gene = HLA.split("*")[0]
        if gene in listofgene:
            return_list.append(HLA)
        else:
            continue
    return return_list


#ใช้โค้ดเดิมใน spliting_fasta.ipynb
def createnewfastafromHLAlist_separatedgene(filename, HLAtoalignment, outputname):
    """ สร้าง HLA ใหม่โดยใช้ HLA list"""
    new_list = []
    for seq_record in SeqIO.parse(filename, "fasta"):  
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        if HLAnomenclature in HLAtoalignment:
            new_list.append(seq_record)
    return SeqIO.write(new_list, outputname, "fasta")

def createnewfastafromGenelist(fastafile, ListofGeneWantedinfile):
    """ สร้าง fasta ใหม่โดยใช้ list gene ที่ต้องการในไฟล์นั้น ๆ """
    new_list = []
    for gene in ListofGeneWantedinfile:
        print(f"Processing for gene: {gene}")
        for seq_record in SeqIO.parse(fastafile, "fasta"):  
            seq_description_list = seq_record.description.split()
            HLAnomenclature = seq_description_list[1]
            GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
            if GenefromHLAnomenclature == gene:
                new_list.append(seq_record)
    SeqIO.write(new_list, f'gene_aligned_{ListofGeneWantedinfile}.fasta', "fasta")
    print(f"Saved for gene: {ListofGeneWantedinfile}")
    print("All Done")

def dropuniqueHLAvalue(fastafile, fieldwanted,dropchars):
    """ ลบ HLA ที่ซ้ำกันออกจาก fasta file โดย field wanted = 1 ถึง 4"""
    new_df = pd.DataFrame(columns=['HLA','field1','field2','field3','field4'])
    seq_record_list = []
    
    total_counts = 0
    for seq_record in SeqIO.parse(fastafile, "fasta"):  
        total_counts += 1
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        GenefromHLAnomenclature = HLAnomenclature.split('*')[1]
        HLAnomenclature_splited = GenefromHLAnomenclature.split(':')

        #Add - if field is empty
        while len(HLAnomenclature_splited) < 4:
            HLAnomenclature_splited.append('-')

        to_add = pd.DataFrame({'HLA': HLAnomenclature,
        'field1':HLAnomenclature_splited[0],
        'field2':HLAnomenclature_splited[1],
        'field3':HLAnomenclature_splited[2],
        'field4':HLAnomenclature_splited[3]}, index=[0])

        new_df = new_df.append(to_add, ignore_index=True)
    new_df = new_df.reset_index(drop=True)
    df_dropped = new_df.drop_duplicates(subset=[f'field{x+1}' for x in range(fieldwanted)], keep='first')

    # Process drop chars
    if dropchars == True:
        to_drop = []
        #counter = 0
        for x in df_dropped.iterrows():
            index = -1
            y = x[-1][index][-1]
            while(y == '-'):
                index -= 1
                y = x[-1][index][-1]
            if y in ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']:
                to_drop.append(x[0])
            #counter += 1
        df_dropped = df_dropped.drop(to_drop)

    # Save file
    index_counter = 0
    for seq_record in SeqIO.parse(fastafile, "fasta"):  
        if index_counter in df_dropped.index:
            seq_record_list.append(seq_record)
        index_counter += 1

    # print(new_df)
    # print(df_dropped)
    # print(df_dropped.index)
    # # print(new_list)
    # print(seq_record_list)
    # f = open(f'{fastafile}_dropdupe{fieldwanted}field_dropchar{str(dropchars)}.txt', "w")
    # f.write()
    # f.close()
    SeqIO.write(seq_record_list, f'{fastafile}_dropdupe{fieldwanted}field_dropchar{str(dropchars)}.fasta', "fasta")
    
    #print(f"Saved for gene: {ListofGeneWantedinfile}")
    print(f"{total_counts} were removed to {len(seq_record_list)} records")
    print("All Done")

#สร้าง csv จาก gene list
def createnewfastafromGenelist_separatedgene_csv(filename, Genetoallignment):
    for gene in Genetoallignment:
        new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
        print(f"Processing for gene: {gene}")
        for seq_record in SeqIO.parse(filename, "fasta"):  
            seq_description_list = seq_record.description.split()
            HLAnomenclature = seq_description_list[1]
            GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
            allele = seq_record.description.split()[1]
            sequence = repr(seq_record.seq).split("'")[1]
            if GenefromHLAnomenclature == gene:
                new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence},ignore_index=True)
        print(f"Saved for gene: {gene}")
        new_csv.to_csv(f'gene_{gene}.csv')
    print("All Done")

# แปลง fasta เป็น csv
def convertfastatocsv(filename):
    """ Input file in fasta format to convert to csv """
    new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
    for seq_record in SeqIO.parse(filename, "fasta"):  
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
        allele = seq_record.description.split()[1]
        sequence = ""
        for x in list(seq_record.seq):
            sequence = sequence+x
        new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence,"Char":len(sequence)},ignore_index=True)
    new_csv.to_csv(f'gene_{GenefromHLAnomenclature}_aligned.csv',index=False)

# def convertfastatocsv_unalligned(filename):
#     """ Input file in fasta format to convert to csv """
#     new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
#     for seq_record in SeqIO.parse(filename, "fasta"):  
#         seq_description_list = seq_record.description.split()
#         HLAnomenclature = seq_description_list[1]
#         GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
#         allele = seq_record.description.split()[1]
#         sequence = ""
#         for x in list(seq_record.seq):
#             sequence = sequence+x
#         new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence,"Char":len(sequence)},ignore_index=True)
#     new_csv.to_csv(f'gene_{GenefromHLAnomenclature}_hla_prot.csv',index=False)


In [2]:
#modify convertfastatocsv() เพื่อแปลง aligned fasta เป้น csv
filenameclass = {1:"../2 - HLA Proteins/gene_filtered_['A', 'B', 'C']_MAFFT_ALIGNED.fasta",
                2:"../2 - HLA Proteins/gene_filtered_['DPB1', 'DQA1', 'DQB1', 'DRB1', 'DRB3', 'DRB4', 'DRB5']_MAFFT_ALIGNED.fasta"}

def convertfastatocsv_2(filename):
    new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
    for seq_record in SeqIO.parse(filename, "fasta"):
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
        allele = seq_record.description.split()[1]
        sequence = ""
        for x in list(seq_record.seq):
            sequence = sequence+x
        new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence},ignore_index=True)
    return new_csv
class1 = convertfastatocsv_2(filenameclass[1])
class2 = convertfastatocsv_2(filenameclass[2])
#new_csv.to_csv(f'gene_class1_aligned.csv',index=False)

In [3]:
#กรอง index แล้วเซฟกลับเข้าใน csv ใหม่
#select appropriate part
# class I -> 140-179 เป็น 225-279
# class 1  -> 50-84 เป็น 87-128
# ใส่ไปได้เลยเพราะ python เริ่มนับ 0 Aliview นับ 1

def selectsequence(df, col_name, x,y):
    new_csv = df.copy()
    for lab,row in new_csv.iterrows():
        new_csv.iloc[lab][col_name] = row[col_name][x:y]
    return new_csv

selected_prot1_class1 = selectsequence(class1, 'Sequence', 86, 128) #alpha1
selected_prot2_class1 = selectsequence(class1, 'Sequence', 224, 279) #alpha2

In [4]:
new_csv = selected_prot1_class1.merge(selected_prot2_class1, how='inner', on='Alleles', suffixes=('_prot1',"_prot2"))

In [5]:
#new_csv = pd.concat([new_csv1,new_csv2], axis=0)
new_csv = new_csv.reset_index()
new_csv
#ปรับให้จำนวน char กันทั้ง column sequence

,index,Alleles,Sequence_prot1,Sequence_prot2
0,0,A*01:01:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...
1,1,A*01:02:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...
2,2,A*01:03:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...
3,3,A*01:06,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY,AQITKRKWEAVHAAEQLRA------------YLEGRCVDGLRRY--...
4,4,A*01:07,P-----WIEQERPEYWDQETR-NVKAHSQTDRENLG-TLRGY,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...
...,...,...,...,...
12540,12540,C*17:67,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...
12541,12541,C*17:68,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...
12542,12542,C*17:69,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...
12543,12543,C*18:16,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY,AQITQRKWEAAREAEQRRA------------YLEGTCVEWLRRY--...


In [6]:
new_csv

,index,Alleles,Sequence_prot1,Sequence_prot2
0,0,A*01:01:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...
1,1,A*01:02:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...
2,2,A*01:03:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...
3,3,A*01:06,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY,AQITKRKWEAVHAAEQLRA------------YLEGRCVDGLRRY--...
4,4,A*01:07,P-----WIEQERPEYWDQETR-NVKAHSQTDRENLG-TLRGY,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...
...,...,...,...,...
12540,12540,C*17:67,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...
12541,12541,C*17:68,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...
12542,12542,C*17:69,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...
12543,12543,C*18:16,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY,AQITQRKWEAAREAEQRRA------------YLEGTCVEWLRRY--...


In [7]:
new_csv.iloc[0,2]

'P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY'

In [8]:
new_csv.iloc[0,3]

'AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY---LENGKETL'

In [9]:
def updatechar(csv, col_name):
    #csv['Char'] = 0
    for lab, row in csv.iterrows():
        new_csv.loc[lab, col_name+'_Char'] = len(row[col_name])
updatechar(new_csv,'Sequence_prot1')
updatechar(new_csv,'Sequence_prot2')

In [10]:
#find col containing _char behind its name
col_containing_char = []
for x in new_csv.columns.tolist():
    if '_Char' in x: #เชี้ย ความรู้ใหม่
        col_containing_char.append(x)

        

In [11]:
col_containing_char

['Sequence_prot1_Char', 'Sequence_prot2_Char']

In [12]:
max_char = int(new_csv[col_containing_char].max().max()) #หา max ของ col ดังกล่าว

#iterate ของแต่ละ col เทียบกับ max_char ถ้ายังไม่ถึงค่า max ให้เติม - ด้านหลังมัน
for x in col_containing_char:
    for lab, row in new_csv.iterrows():
        if row[x] < max_char:
            #old_sequence = row[x[:-5]]
            new_sequence = row[x[:-5]] + '-'*(int(max_char-row[x]))
            new_csv.loc[lab,x[:-5]] = new_sequence

# for lab,row in new_csv.iterrows():
#     if len(row) < max_char:
#         old_sequence = row['Sequence']
#         new_sequence = row['Sequence'] + '-'*(int(max_char-row['Char'])) 
#         new_csv.loc[lab,'Sequence'] = new_sequence
new_csv

,index,Alleles,Sequence_prot1,Sequence_prot2,Sequence_prot1_Char,Sequence_prot2_Char
0,0,A*01:01:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,42.0,55.0
1,1,A*01:02:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,42.0,55.0
2,2,A*01:03:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,42.0,55.0
3,3,A*01:06,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQLRA------------YLEGRCVDGLRRY--...,42.0,55.0
4,4,A*01:07,P-----WIEQERPEYWDQETR-NVKAHSQTDRENLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,42.0,55.0
...,...,...,...,...,...,...
12540,12540,C*17:67,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...,42.0,55.0
12541,12541,C*17:68,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...,42.0,55.0
12542,12542,C*17:69,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...,42.0,55.0
12543,12543,C*18:16,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQITQRKWEAAREAEQRRA------------YLEGTCVEWLRRY--...,42.0,55.0


In [13]:
updatechar(new_csv,'Sequence_prot1')
updatechar(new_csv,'Sequence_prot2')

In [14]:
new_csv

,index,Alleles,Sequence_prot1,Sequence_prot2,Sequence_prot1_Char,Sequence_prot2_Char
0,0,A*01:01:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0
1,1,A*01:02:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0
2,2,A*01:03:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0
3,3,A*01:06,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQLRA------------YLEGRCVDGLRRY--...,55.0,55.0
4,4,A*01:07,P-----WIEQERPEYWDQETR-NVKAHSQTDRENLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0
...,...,...,...,...,...,...
12540,12540,C*17:67,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...,55.0,55.0
12541,12541,C*17:68,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...,55.0,55.0
12542,12542,C*17:69,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...,55.0,55.0
12543,12543,C*18:16,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQITQRKWEAAREAEQRRA------------YLEGTCVEWLRRY--...,55.0,55.0


In [15]:
new_col = []
for lab,row in new_csv.iterrows():
    alleles = row['Alleles'].split(':')
    new_alleles = alleles[0:2]
    new_alleles = ":".join(new_alleles)
    new_col.append(new_alleles)
new_col

['A*01:01',
 'A*01:02',
 'A*01:03',
 'A*01:06',
 'A*01:07',
 'A*01:08',
 'A*01:09',
 'A*01:10',
 'A*01:12',
 'A*01:13',
 'A*01:14',
 'A*01:17',
 'A*01:19',
 'A*01:20',
 'A*01:21',
 'A*01:23',
 'A*01:24',
 'A*01:25',
 'A*01:26',
 'A*01:28',
 'A*01:29',
 'A*01:30',
 'A*01:32',
 'A*01:33',
 'A*01:35',
 'A*01:36',
 'A*01:37',
 'A*01:38',
 'A*01:39',
 'A*01:40',
 'A*01:41',
 'A*01:42',
 'A*01:43',
 'A*01:44',
 'A*01:45',
 'A*01:46',
 'A*01:47',
 'A*01:48',
 'A*01:49',
 'A*01:50',
 'A*01:51',
 'A*01:54',
 'A*01:55',
 'A*01:58',
 'A*01:59',
 'A*01:60',
 'A*01:61',
 'A*01:62',
 'A*01:63',
 'A*01:64',
 'A*01:65',
 'A*01:66',
 'A*01:67',
 'A*01:68',
 'A*01:69',
 'A*01:70',
 'A*01:71',
 'A*01:72',
 'A*01:73',
 'A*01:74',
 'A*01:75',
 'A*01:76',
 'A*01:77',
 'A*01:78',
 'A*01:79',
 'A*01:80',
 'A*01:81',
 'A*01:82',
 'A*01:83',
 'A*01:84',
 'A*01:85',
 'A*01:86',
 'A*01:88',
 'A*01:89',
 'A*01:90',
 'A*01:91',
 'A*01:92',
 'A*01:93',
 'A*01:94',
 'A*01:95',
 'A*01:96',
 'A*01:97',
 'A*01:98',
 'A*

In [16]:
new_csv["new_alleles"] = new_col
new_csv

,index,Alleles,Sequence_prot1,Sequence_prot2,Sequence_prot1_Char,Sequence_prot2_Char,new_alleles
0,0,A*01:01:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0,A*01:01
1,1,A*01:02:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0,A*01:02
2,2,A*01:03:01:01,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0,A*01:03
3,3,A*01:06,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQLRA------------YLEGRCVDGLRRY--...,55.0,55.0,A*01:06
4,4,A*01:07,P-----WIEQERPEYWDQETR-NVKAHSQTDRENLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0,A*01:07
...,...,...,...,...,...,...,...
12540,12540,C*17:67,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...,55.0,55.0,C*17:67
12541,12541,C*17:68,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...,55.0,55.0,C*17:68
12542,12542,C*17:69,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQISQRKLEAAREAEQLRA------------YLEGECVEWLRGY--...,55.0,55.0,C*17:69
12543,12543,C*18:16,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQITQRKWEAAREAEQRRA------------YLEGTCVEWLRRY--...,55.0,55.0,C*18:16


# ตรงนี้ต้องไปรันใน 1 C&T_FilteringRenaming.ipynb ก่อนให้ได้ไฟล์ 1 Dataset_Filtered.csv

In [17]:
#sort เอาแค่ alleles ที่ตรงกันกับฐานฐานข้อมูล
df_dataset = pd.read_csv('1 Dataset_Filtered.csv')
dataset_alleles_list = df_dataset['Allele'].unique().tolist()

In [18]:
inboth = set(dataset_alleles_list) & set(new_col)

In [19]:
new_csv_inboth = new_csv[new_csv['new_alleles'].isin(inboth)]
#new_csv_inboth = new_csv_inboth[['new_alleles','Char','Sequence']]
new_csv_inboth = new_csv_inboth.drop(columns=['Alleles','index'])
new_csv_inboth = new_csv_inboth.reset_index(drop=True)
new_csv_inboth

,Sequence_prot1,Sequence_prot2,Sequence_prot1_Char,Sequence_prot2_Char,new_alleles
0,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0,A*01:01
1,P-----WIEQEGPEYWDGETR-KVKAHSQTHRVDLG-TLRGY----...,AQTTKHKWEAAHVAEQLRA------------YLEGTCVEWLRRY--...,55.0,55.0,A*02:01
2,P-----WIEQEGPEYWDGETR-KVKAHSQTHRVDLG-TLRGY----...,AQTTKHKWETAHEAEQWRA------------YLEGTCVEWLRRY--...,55.0,55.0,A*02:03
3,P-----WIEQEGPEYWDGETR-KVKAHSQTHRVDLG-TLRGY----...,AQTTKHKWEAAHVAEQLRA------------YLEGTCVEWLRRY--...,55.0,55.0,A*02:06
4,P-----WIEQEGPEYWDGETR-KVKAHSQTHRVDLG-TLRGY----...,AQTTKHKWEAAHVAEQLRA------------YLEGTCVEWLRRY--...,55.0,55.0,A*02:07
...,...,...,...,...,...
113,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQITQRKWEAAREAEQWRA------------YLEGTCVEWLRRY--...,55.0,55.0,C*06:02
114,P-----WVEQEGPEYWDRETQ-NYKRQAQADRVSLR-NLRGY----...,AQITQRKLEAARAAEQLRA------------YLEGTCVEWLRRY--...,55.0,55.0,C*07:01
115,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVSLR-NLRGY----...,AQITQRKWEAAREAEQWRA------------YLEGTCVEWLRRY--...,55.0,55.0,C*12:02
116,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVSLR-NLRGY----...,AQITQRKWEAAREAEQWRA------------YLEGTCVEWLRRY--...,55.0,55.0,C*12:03


In [20]:
total_char = []
for lab, row in new_csv_inboth.iterrows():
    list_toadd = []
    for x in col_containing_char:
        list_toadd = list_toadd + [x for x in row[x[:-5]]]
    total_char = list_toadd + total_char
amino_list = list(set(total_char))

def quicksort(lst): 
    if not lst:
        return []
    return (quicksort([x for x in lst[1:] if x <  lst[0]])
            + [lst[0]] +
            quicksort([x for x in lst[1:] if x >= lst[0]]))

amino_list = quicksort(amino_list)

In [21]:
amino_list

['-',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y']

In [22]:
final_dataset = new_csv_inboth.copy()

for col_name in col_containing_char:
    for x in range(max_char): #max_char
        new_amino_list = []
        #แก้ชื่อ col ให้เป้นไปตามลำดับของ sequence
        for i in range(len(amino_list)):
            new_amino_list.append("p"+str(col_containing_char.index(col_name)+1) + "-" + str(x) + "-" + amino_list[i])

        #สร้าง dataframe ใหม่ด้วย col ที่สร้างขึ้นมา
        to_add = pd.DataFrame(columns=new_amino_list)

        for lab,row in final_dataset.iterrows():
            char_tocheck = row[col_name[:-5]][x]
            to_add.loc[lab,"p"+str(col_containing_char.index(col_name)+1) + "-" + str(x) + "-" + amino_list[i]] = 1
            to_add = to_add.fillna(0).astype(int)
        final_dataset = pd.concat([final_dataset,to_add], axis=1)
        print(f"sequence no. {x} done, of {col_name}")


sequence no. 0 done, of Sequence_prot1_Char
sequence no. 1 done, of Sequence_prot1_Char
sequence no. 2 done, of Sequence_prot1_Char
sequence no. 3 done, of Sequence_prot1_Char
sequence no. 4 done, of Sequence_prot1_Char
sequence no. 5 done, of Sequence_prot1_Char
sequence no. 6 done, of Sequence_prot1_Char
sequence no. 7 done, of Sequence_prot1_Char
sequence no. 8 done, of Sequence_prot1_Char
sequence no. 9 done, of Sequence_prot1_Char
sequence no. 10 done, of Sequence_prot1_Char
sequence no. 11 done, of Sequence_prot1_Char
sequence no. 12 done, of Sequence_prot1_Char
sequence no. 13 done, of Sequence_prot1_Char
sequence no. 14 done, of Sequence_prot1_Char
sequence no. 15 done, of Sequence_prot1_Char
sequence no. 16 done, of Sequence_prot1_Char
sequence no. 17 done, of Sequence_prot1_Char
sequence no. 18 done, of Sequence_prot1_Char
sequence no. 19 done, of Sequence_prot1_Char
sequence no. 20 done, of Sequence_prot1_Char
sequence no. 21 done, of Sequence_prot1_Char
sequence no. 22 done

In [23]:
final_dataset

,Sequence_prot1,Sequence_prot2,Sequence_prot1_Char,Sequence_prot2_Char,new_alleles,p1-0--,p1-0-A,p1-0-C,p1-0-D,p1-0-E,...,p2-54-M,p2-54-N,p2-54-P,p2-54-Q,p2-54-R,p2-54-S,p2-54-T,p2-54-V,p2-54-W,p2-54-Y
0,P-----WIEQEGPEYWDQETR-NMKAHSQTDRANLG-TLRGY----...,AQITKRKWEAVHAAEQRRV------------YLEGRCVDGLRRY--...,55.0,55.0,A*01:01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,P-----WIEQEGPEYWDGETR-KVKAHSQTHRVDLG-TLRGY----...,AQTTKHKWEAAHVAEQLRA------------YLEGTCVEWLRRY--...,55.0,55.0,A*02:01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,P-----WIEQEGPEYWDGETR-KVKAHSQTHRVDLG-TLRGY----...,AQTTKHKWETAHEAEQWRA------------YLEGTCVEWLRRY--...,55.0,55.0,A*02:03,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,P-----WIEQEGPEYWDGETR-KVKAHSQTHRVDLG-TLRGY----...,AQTTKHKWEAAHVAEQLRA------------YLEGTCVEWLRRY--...,55.0,55.0,A*02:06,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,P-----WIEQEGPEYWDGETR-KVKAHSQTHRVDLG-TLRGY----...,AQTTKHKWEAAHVAEQLRA------------YLEGTCVEWLRRY--...,55.0,55.0,A*02:07,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVNLR-KLRGY----...,AQITQRKWEAAREAEQWRA------------YLEGTCVEWLRRY--...,55.0,55.0,C*06:02,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
114,P-----WVEQEGPEYWDRETQ-NYKRQAQADRVSLR-NLRGY----...,AQITQRKLEAARAAEQLRA------------YLEGTCVEWLRRY--...,55.0,55.0,C*07:01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
115,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVSLR-NLRGY----...,AQITQRKWEAAREAEQWRA------------YLEGTCVEWLRRY--...,55.0,55.0,C*12:02,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
116,P-----WVEQEGPEYWDRETQ-KYKRQAQADRVSLR-NLRGY----...,AQITQRKWEAAREAEQWRA------------YLEGTCVEWLRRY--...,55.0,55.0,C*12:03,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
col_containing_char.index(col_containing_char[0])

0

In [25]:
final_dataset.to_csv('0 Selected_Protein_Seq.csv', index=None)

# EDA

In [36]:
# checkcheck = pd.concat([class1,class2], axis=0)
# checkcheck

In [37]:
# new_col = []
# for lab,row in checkcheck.iterrows():
#     alleles = row['Alleles'].split(':')
#     new_alleles = alleles[0:2]
#     new_alleles = ":".join(new_alleles)
#     new_col.append(new_alleles)
# new_col

In [38]:
# inboth = set(dataset_alleles_list) & set(new_col)

In [39]:
# inboth

In [40]:
# list_tosort = []
# for x in list(inboth):
#     if x.split("*")[0][0] == "D":
#         #print(x.split("*")[0])
#         list_tosort.append(x)
#         print(x)